# Introduction to the ACCESS-NRI Intake catalog

**Aims**: This tutorial will introduce the ACCESS-NRI Intake catalog and show you how to use it to find and load model data for analysis

**Project membership requirements**:

 - If using the `xp65` conda environment: `xp65`, `dk92`, `fs38`, `p73`, `ik11` and `oi10`
 - If using the `hh5` conda environment: as above but also `hh5`


The ACCESS-NRI Intake Catalog is curated by ACCESS-NRI with further information on its <a href="https://access-nri-intake-catalog.readthedocs.io/en/latest/index.html" target="_blank">documentation website</a> and <a href="https://access-hive.org.au/model_evaluation/data/model_catalogs/" target="_blank">this ACCESS-Hive page</a>.

----

# Exercise 1

In [24]:
import intake

In [25]:
catalog = intake.cat.access_nri

View the catalog

In [26]:
catalog

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v13_ryf9091,{ACCESS-OM2},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},"{seaIce, ocean}","{3mon, 3hr, fx, 1day, 1mon}","{temp_rivermix, total_ocean_swflx_vis, congel_m, ANGLE, pot_rho_0, sfc_hflux_from_runoff, aicen_m, u_dot_grad_vert_pv, bih_fric_u, ty_trans_int_z, tau_y, total_ocean_river_heat, TLAT, wfimelt, rho..."
01deg_jra55v140_iaf,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{fx, 1day, 1mon}","{congel, diff_cbt_t, total_ocean_swflx_vis, ANGLE, congel_m, pot_rho_0, sfc_hflux_from_runoff, dvidtd_m, aicen_m, u_dot_grad_vert_pv, vorticity_z, bih_fric_u, dvidtd, daidtt_m, tau_y, total_ocean_..."
01deg_jra55v140_iaf_cycle2,{ACCESS-OM2},{Cycle 2 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{fx, 1day, 1mon}","{congel, diff_cbt_t, total_ocean_swflx_vis, ANGLE, congel_m, pot_rho_0, sfc_hflux_from_runoff, dvidtd_m, Tinz_m, aicen_m, u_dot_grad_vert_pv, divu, Sinz_m, bih_fric_u, dvidtd, daidtt_m, Sinz, tau_..."
01deg_jra55v140_iaf_cycle3,{ACCESS-OM2},{Cycle 3 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{fx, 1day, 1mon}","{congel, diff_cbt_t, total_ocean_swflx_vis, ANGLE, congel_m, pot_rho_0, sfc_hflux_from_runoff, dvidtd_m, Tinz_m, aicen_m, divu, Sinz_m, ty_trans_int_z, dvidtd, daidtt_m, Sinz, tau_y, total_ocean_r..."
01deg_jra55v140_iaf_cycle4,{ACCESS-OM2},{Cycle 4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{6hr, 3hr, fx, 1day, 1mon}","{Tinz_m, npp_intmld, Sinz_m, tau_y, rhoave, sfc_salt_flux_ice, mld, surface_pot_temp, temp_yflux_adv_int_z, surface_o2, sea_level, divu_h, total_ocean_mh_flux, surface_pot_temp_min, geolat_t, tx_t..."
01deg_jra55v140_iaf_cycle4_jra55v150_extension,{ACCESS-OM2},{Extensions of cycle 4 of 0.1 degree ACCESS-OM2 + WOMBAT BGC global model configuration with JRA55-do v1.5.0 and v1.5.0.1 interannual forcing},"{seaIce, ocean}","{subhr, fx, 1day, 1mon}","{congel, algal_N, diff_cbt_t, total_ocean_swflx_vis, ANGLE, congel_m, src05, skl_Nit, pot_rho_0, sfc_hflux_from_runoff, npp_intmld, dvidtd_m, adic, o2, pprod_gross_2d, divu, aicen_m, ty_trans_int_..."
01deg_jra55v150_iaf_cycle1,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.5.0 OMIP2 interannual forcing},"{seaIce, ocean}","{fx, 1day, 1mon}","{diff_cbt_t, sfc_hflux_from_runoff, ty_trans_int_z, tau_y, TLAT, wfimelt, mld_max, sfc_salt_flux_ice, mld, ty_trans_nrho_submeso, surface_pot_temp, temp_yflux_adv_int_z, temp, sea_level, salt, dzt..."
025deg_era5_iaf,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with ERA5 interannual\nforcing (1980-2021)},"{seaIce, ocean}","{fx, 1day, 1mon}","{congel, total_ocean_swflx_vis, ANGLE, congel_m, pot_rho_0, sfc_hflux_from_runoff, dvidtd_m, aicen_m, ty_trans_int_z, dvidtd, daidtt_m, tau_y, total_ocean_river_heat, strcorx_m, TLAT, rhoave, wfim..."
025deg_era5_ryf,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with ERA5 RYF9091 repeat\nyear forcing (May 1990 to Apr 1991)},"{seaIce, ocean}","{fx, 1day, 1mon}","{congel, diff_cbt_t, total_ocean_swflx_vis, ANGLE, congel_m, pot_rho_0, sfc_hflux_from_runoff, dvidtd_m, aicen_m, ty_trans_int_z, dvidtd, daidtt_m, tau_y, total_ocean_river_heat, strcorx_m, TLAT, ..."


Search the catalog by any of its columns, e.g.

```python
catalog.search(model='ACCESS-OM2')
catalog.search(model=..., frequency=..., variable=...)
```

Select a single datastore using it's <b>name</b>, e.g.

```python
datastore = catalog['01deg_jra...']
```

If your search returns a single experiment:

```python
datastore = catalog.search(name=...).to_source()
```


View the datastore

```python
datastore
```
and

```python
datastore.df
```

Find a variable

```python
datastore.search(variable=..., frequency=...)
```
search by the columns in the datastore.

Refine your search to reach <b>1 dataset</b>, using `.keys()` and `.keys_info()` to assist

# Exercise 2

Save your search returning one dataset

```python
search=datastore.search(...)
```

(If you need to catch up, you could use:
```python
search = catalog.search(name='025deg_jra55_iaf_omip2_cycle6', variable = 'sst', frequency='1mon')
```
)

Start a dask cluster

In [27]:
from dask.distributed import Client

In [28]:
client = Client(threads_per_worker=1)

In [29]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 12,Total memory: 46.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34203,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 46.00 GiB
Comm: tcp://127.0.0.1:42577,Total threads: 1
Dashboard: /proxy/34935/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:42987,


Open your dataset

```python
ds = search.to_dask()
```

and check `ds` contains the variable you expect

Try a search which returns more than one variable:
``` python
search = catalog['01deg_jra55v140_iaf'].search(variable=['temp_surface_ave', 'salt_surface_ave'])
search
```

Can we open that with `to_dask()` ?

Repeat for 

```python
search = catalog['01deg_jra55v140_iaf'].search(variable=['surface_salt', 'surface_temp'], frequency='1mon')
search
```

Use `search.keys()` and `search.keys_info()` if needed

For the second search, we can combine after opening:


``` python
import xarray as xr
search = catalog['01deg_jra55v140_iaf'].search(variable=['surface_salt', 'surface_temp'], frequency='1mon')
ds_dict = search.to_dataset_dict()
ds_dict
ds = xr.merge(
    ds_dict.values(), 
)
```

Try a search with more than one experiment
```python
search = catalog.search(name='01deg_jra55v140_iaf_cycle4.*')
search
```

With more than one experiment, we need to use `.to_source_dict()`
```python
datastore_dict = catalog.search(name='01deg_jra55v140_iaf_cycle4.*').to_source_dict()

This returns a dictionary or intake-esm datastores, we can open them in a loop, and then concatenate them:

```python

dataset_dict = {
    name: datastore.search(variable="temp_surface_ave").to_dask()
    for name, datastore in datastore_dict.items()
}

ds = xr.concat(
    dataset_dict.values(),
    dim='time'
)
```

In [32]:
import xarray as xr

In [36]:
search = catalog.search(name='01deg_jra55v140_iaf_cycle4.*')

In [37]:
search

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v140_iaf_cycle4,{ACCESS-OM2},{Cycle 4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{6hr, 3hr, fx, 1day, 1mon}","{Tinz_m, npp_intmld, Sinz_m, tau_y, rhoave, sfc_salt_flux_ice, mld, surface_pot_temp, temp_yflux_adv_int_z, surface_o2, sea_level, divu_h, total_ocean_mh_flux, surface_pot_temp_min, geolat_t, tx_t..."
01deg_jra55v140_iaf_cycle4_jra55v150_extension,{ACCESS-OM2},{Extensions of cycle 4 of 0.1 degree ACCESS-OM2 + WOMBAT BGC global model configuration with JRA55-do v1.5.0 and v1.5.0.1 interannual forcing},"{seaIce, ocean}","{subhr, fx, 1day, 1mon}","{congel, algal_N, diff_cbt_t, total_ocean_swflx_vis, ANGLE, congel_m, src05, skl_Nit, pot_rho_0, sfc_hflux_from_runoff, npp_intmld, dvidtd_m, adic, o2, pprod_gross_2d, divu, aicen_m, ty_trans_int_..."


In [38]:
datastore_dict = search.to_source_dict()

In [39]:
dataset_dict = {
    name: datastore.search(variable="temp_surface_ave").to_dask()
    for name, datastore in datastore_dict.items()
}

ds = xr.concat(
    dataset_dict.values(),
    dim='time'
)

In [40]:
ds

<xarray.Dataset> Size: 384kB
Dimensions:           (time: 23984, scalar_axis: 1)
Coordinates:
  * scalar_axis       (scalar_axis) float64 8B 0.0
  * time              (time) datetime64[ns] 192kB 2019-01-02 ... 2019-01-01
Data variables:
    temp_surface_ave  (time, scalar_axis) float64 192kB dask.array<chunksize=(1, 1), meta=np.ndarray>
Attributes: (12/14)
    title:                                    ACCESS-OM2-01
    grid_type:                                mosaic
    grid_tile:                                1
    intake_esm_vars:                          ['temp_surface_ave']
    intake_esm_attrs:realm:                   ocean
    intake_esm_attrs:variable:                eta_global,ke_tot,pe_tot,rhoave...
    ...                                       ...
    intake_esm_attrs:variable_standard_name:  ,,,,sea_water_salinity,,sea_wat...
    intake_esm_attrs:variable_cell_methods:   time: point,time: point,time: p...
    intake_esm_attrs:variable_units:          meter,10^15 Joules,10^15 Joules...
    intake_esm_attrs:file_id:                 ocean_scalar_1_daily_ym_XXXX_XX
    intake_esm_attrs:_data_format_:           netcdf
    intake_esm_dataset_key:                   ocean_scalar_1_daily_ym_XXXX_XX...

In [23]:
client.close()